# 载入数据

In [1]:
import pymysql
# 取得需要进行分析的数据
try:
    connection = pymysql.connect(host='116.63.167.124',  # host属性
                                 user='root',  # 用户名
                                 password='',  # 此处填登录数据库的密码
                                 db='job_database'  # 数据库名
                                 )
    # 使用 cursor() 方法创建一个游标对象 cursor
    cursor = connection.cursor()
    # 使用 execute()  方法执行 SQL 查询
    cursor.execute("SELECT * FROM job_table WHERE job_name like '%%' and job_academic like '%%' and job_city like '%%'")
    # 使用 fetchall() 方法获取全部数据.
    data = cursor.fetchall()
    connection.close()
except:
    # 发生错误时回滚
    print("错误")
if len(data) > 0:
    print(len(data))

27628


# 数据预处理

##### 定义清理停用词与分词函数

In [2]:
import jieba
import re
import numpy as np
import gensim
from gensim.models import Word2Vec
# 读取停用词列表
def get_stopword_list(file):
    with open(file, 'r', encoding='utf-8') as f:    # 
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list


# 分词 然后清除停用词语
def clean_stopword(string, stopword_list):
    result = ''
    # stopword_list.append('erp')
    if string not in stopword_list:
    # 将所有不在停用词表中的词汇留下并将其中所有的大写字母转换成小写便于统计
        result += string.lower()
    return result

In [3]:
import re
# 载入data中的detail_result列并将所有的数据进行载入成一列数据列表之后
def update(one_data):
    line = []
    if one_data is not None:
        temp = re.finditer('(?!([0-9. -]))[\u4e00-\u9fa5/a-zA-Z0-9 +.-/#-]+', one_data)
        for word in temp:
            # 去除chatgpt返回的以技术开头的词汇
            if not word.group().startswith("技"):
                line.append(word.group().replace(' ',''))
    return line

In [4]:
def clean(one_line):
    # 可在停用词表中添加自己不想留下的词汇
    stopword_file = './hit_stopwords.txt'
    stopword_list = get_stopword_list(stopword_file)    # 获得停用词列表
    result = []
    # 本行数据非空
    if one_line is not None:  
        for temp in one_line:
            result.append(clean_stopword(temp, stopword_list))
    # 去除去除停用词之后的空值
    one_line = [temp for temp in result if temp]
    return one_line

## 定义清理detail_result函数

In [5]:
# 以下为使用训练好的模型数据清洗的全过程
# 清洗数据详情的函数,传入值为待清洗的该列数据
def clean_detail(one_data):
    # 加载jieba词典，已经去除了使用jieba分词所以不再需要了
    # jieba.load_userdict('./dict.txt')
    # 得出训练词汇列表
    line = update(one_data) 
    # 清理停用词
    line = clean(line)
    final_result = []
    for word in line:
        try:
            # 好像用不用这个无所谓，因为我并没有输入词库进行对比，导致用之后反而会使部分出现的次数比较多的词汇由于与其他词汇的相关性较弱导致不能将该词汇统计到最终结果之中
            # temp = model.wv.most_similar(word, topn = 1)[0][1]
            if temp in word_list:
                final_result.append(word)
        except:
            continue
    clean_detail = final_result
    return line

In [6]:
# 清洗工作年限要求
import numpy as np
def clean_years(job_years):
    # 待清理数据中间存放列表
    middle = re.findall("[\d]+",job_years)
    result = ([int(num) for num in middle])
# 返回最小值工作年限要求的
    return min(result,default=0)

# 在Python中，内置函数 min() 可以用于获取一组值中的最小值。如果您使用 Anaconda 平台或其他 Python 集成开发环境，并且遇到了无法使用 min() 函数的问题，可能是以下几个原因导致：
# 可能您的代码中有其他的变量或函数名与 min 冲突，导致 min() 函数无法被正确调用。您可以尝试使用 builtins.min() 显式地调用 min() 函数，或者修改其他变量或函数名，避免冲突。
# 可能您的代码中使用了 NumPy 或其他科学计算库，这些库中也有名为 min() 的函数，可能与 Python 内置的 min() 函数产生冲突。您可以尝试使用 np.min() 等库函数来代替 Python 内置的 min() 函数。
# 可能您的 Python 环境存在问题，例如安装不完整或损坏等。您可以尝试重新安装 Anaconda 或重新创建一个新的 Python 虚拟环境，并确保所有必需的库和组件都已正确安装。
# 总之，如果您无法使用 min() 函数，您应该检查是否存在命名冲突或环境问题，并采取相应的措施来解决。

In [7]:
def clean_acadamic(job_academic):
    if not job_academic:
        return '无学历要求'
    else:
        return job_academic

In [8]:
# 清洗薪资待遇
def clean_compensation(job_compensation):
    # 若其中的格式为2-8千则修改为2000-8千
    if re.search("[\d|\.]+[^千&&万]*-.+千", job_compensation):
        middle = re.search("[\d|\.]+[^千&&万]*-.+千", job_compensation)
        start = re.search('[\d|\.]+', middle[0])[0]
        fw = str(int(float(re.search('[\d|\.]+', middle[0])[0])*1000))
        job_compensation = job_compensation.replace(start, fw, 1)
    # 若其中的格式为2-8万则修改为20000-8万
    if re.search("[\d|\.]+[^千&&万]*-.+万", job_compensation):
        middle = re.search("[[\d|\.]+[^千&&万]*-.+万", job_compensation)
        start = re.search('[\d|\.]+', middle[0])[0]
        fw = str(int(float(re.search('[\d|\.]+', middle[0])[0])*10000))
        job_compensation = job_compensation.replace(start, fw, 1)
    # 8千修改为8000
    if re.findall("[\d|\.]+千", job_compensation):
        for qian in re.findall("[\d|\.]+千", job_compensation):
            k = str(int(float(re.search('[\d|\.]+', qian)[0])*1000))
            job_compensation = job_compensation.replace(qian, k, 1)
    # 8万修改为80000
    if re.findall("[\d|\.]+万", job_compensation):
        for wan in re.findall("[\d|\.]+万", job_compensation):
            w = str(int(float(re.search('[\d|\.]+', wan)[0])*10000))
            job_compensation = job_compensation.replace(wan, w, 1)
    # 将/年修改为默认的月
    if re.search("/年", job_compensation):
        job_compensation = job_compensation.replace("/年", "")
        for num in re.findall("[\d|\.]+", job_compensation):
            job_compensation = job_compensation.replace(str(num), str(int(float(num)/12)), 1)
    # 将/天修改为默认的月
    if re.search("/年", job_compensation):
        job_compensation = job_compensation.replace("/年", "")
        for num in re.findall("[\d|\.]+", job_compensation):
            job_compensation = job_compensation.replace(str(num), str(int(float(num)*30)), 1)
    return job_compensation

In [9]:
# 清洗公司行业
def clean_industry(company_industry):
    list = company_industry.split(",")
    return list

In [10]:
import json
from jsonpath import jsonpath
# 读打开文件
with open('./cityData.json', encoding='utf-8') as a:
        # 读取文件
        json_file = json.load(a)
def find_province(location): 
    temp = jsonpath(json_file, '$.data[?(@.label)].label')
    # 设置跳出外层循环的标记
    lock = False
    for i in range(len(temp)):
        ret = jsonpath(json_file, '$.data[{}]..label'.format(i))
        for word in ret:
            # 找到一个在此省份中的城市则跳出循环
            if location in word:
                lock = True
                break
        if lock is True:
            break
    return temp[i]

In [11]:
# 获得省份信息
def get_province(job_city):
    # 匹配工作城市中的第一个城市名
    list = re.findall('[^-]+', job_city)[0]
    return find_province(list)

In [12]:
import pandas as pd
df = pd.DataFrame(list(data))
df.rename(columns={0:"数据唯一标识符",1:"职位名称",2:"职位详情",3:"技能点",4:"学历要求",5:"工作经验要求",6:"福利",7:"薪资待遇",8:"工作城市",9:"工作地址",10:"招聘公司",11:"公司类型",12:"公司行业",13:"发布时间",14:"招聘数据链接",15:"数据来源网站",16:"数据所在模块", 17: "技能要求"}, inplace=True)

# 修改列名

# 删除不参与分析的列
df = df.drop(['数据唯一标识符','技能点','福利','工作地址','招聘数据链接','数据来源网站'], axis=1)
# len(df["职位详情"])
df['工作经验要求'].fillna("无需经验", inplace = True)
df['学历要求'].fillna("无学历要求", inplace = True)
df['薪资待遇'].fillna(0, inplace = True)
df['公司行业'].fillna("None", inplace = True)
df['技能要求'].fillna("None", inplace = True)

for i in range(len(df["职位详情"])):
    df["职位详情"][i] = clean_detail(df["技能要求"][i])
    df["工作经验要求"][i] = clean_years(df["工作经验要求"][i])
    df["学历要求"][i] = clean_acadamic(df["学历要求"][i])
    df["薪资待遇"][i] = clean_compensation(df["薪资待遇"][i])
    df["公司行业"][i] = clean_industry(df["公司行业"][i])
df = pd.concat([df, pd.DataFrame(columns=['工作所在省份'])], sort=False)
for i in range(len(df["工作城市"])):
    df["工作所在省份"][i] = get_province(df["工作城市"][i])
df = df.drop(['技能要求'], axis=1)


C:\Users\Gimes\AppData\Local\Temp\ipykernel_3724\1529559327.py:4: FutureWarning: Possible set intersection at position 11
  if re.search("[\d|\.]+[^千&&万]*-.+千", job_compensation):
C:\Users\Gimes\AppData\Local\Temp\ipykernel_3724\1529559327.py:10: FutureWarning: Possible set intersection at position 11
  if re.search("[\d|\.]+[^千&&万]*-.+万", job_compensation):
C:\Users\Gimes\AppData\Local\Temp\ipykernel_3724\1529559327.py:11: FutureWarning: Possible nested set at position 1
  middle = re.search("[[\d|\.]+[^千&&万]*-.+万", job_compensation)
C:\Users\Gimes\AppData\Local\Temp\ipykernel_3724\1529559327.py:11: FutureWarning: Possible set intersection at position 12
  middle = re.search("[[\d|\.]+[^千&&万]*-.+万", job_compensation)


In [13]:
display(df)

,职位名称,职位详情,学历要求,工作经验要求,薪资待遇,工作城市,招聘公司,公司类型,公司行业,发布时间,数据所在模块,工作所在省份
0,高级软件工程师,"[c++, java, 主流数据库设计和开发, mes, wms, 低代码平台设计, jav...",本科,5,20833-25000,宁波,宁波伟立机器人科技股份有限公司,民营,"[仪器仪表/工业自动化, 机械/设备/重工]",02-18,高级软件工程师,浙江省
1,高级软件工程师,"[电子线路设计, 汇编语言, arm单片机]",本科,1,10000-18000,深圳-宝安区,新丰电器（深圳）有限公司,外资（非欧美）,[家具/家电/玩具/礼品],02-18,高级软件工程师,广东省
2,Java高级开发工程师,"[java, springmvc, springboot, mybatis, restful...",本科,5,15000-20000·15薪,成都-天府新区,商飞软件有限公司,国企,"[航天/航空, 计算机软件]",02-18,高级软件工程师,四川省
3,运动控制算法工程师,"[差速驱动轮式, 汽车底盘轮式, 动力学建模, 控制系统建模, 算法仿真, 基础运动控制, ...",本科,3,15000-30000,深圳-龙华区,深圳博鹏智能科技有限公司,民营,"[电子技术/半导体/集成电路, 机械/设备/重工]",02-18,高级软件工程师,广东省
4,交换机软件专家,"[交换机, 交换机产品, 架构设计, 组网方案设计, 关键技术, 硬件适配, 芯片问题, 排...",本科,3,30000-60000,深圳-南山区,广州市吉灏电子有限公司,民营,"[计算机软件, 电子技术/半导体/集成电路]",02-18,高级软件工程师,广东省
...,...,...,...,...,...,...,...,...,...,...,...,...
27623,软件工程师,"[c++, 三维互动仿真, 数学等资历, 无加班, 带薪年假, 地铁16号线, 班车接送, ...",本科,2,8000-10000,上海-浦东新区,上海数林软件有限公司,民营,"[计算机软件, 教育/培训/院校]",03-11,软件工程师,上海市
27624,APP开发工程师（网络及流动应用程序),"[reactjs, reactnative, mysql, python, django, ...",本科,1,20000-30000,深圳-罗湖区,惠记环保工程（上海）有限公司,外资（非欧美）,[建筑/建材/工程],03-11,软件工程师,广东省
27625,IT Engineer 软件工程师,[none],本科,1,6000-9000,苏州,太极半导体（苏州）有限公司,国企,[电子技术/半导体/集成电路],03-20,软件工程师,江苏省
27626,接入网关研发工程师/资深专家-深圳 (职位编号：7211361769145731388),[none],本科,0,40000-80000,深圳,字节跳动,民营,"[互联网/电子商务, 计算机软件]",03-20,软件工程师,广东省


In [16]:
df.to_excel("job_detail.xlsx")

In [17]:
groups = df.groupby('学历要求')
for group_name,groups in groups:
    print(group_name)
    print(groups['职位名称'].count())

中技/中专
225
初中及以下
6
博士
90
大专
7108
无学历要求
326
本科
17818
硕士
1885
高中
170


In [18]:
print(df.isnull().sum())

职位名称      0
职位详情      0
学历要求      0
工作经验要求    0
薪资待遇      0
工作城市      0
招聘公司      0
公司类型      0
公司行业      0
发布时间      0
数据所在模块    0
工作所在省份    0
dtype: int64
